In [285]:
import pandas
import psycopg2

In [286]:
# connect database with psycopg2
host = 'localhost'
port = '5432'
dbname = 'mydatabase'
username = 'maciek'
pwd = 'dbhaslo'

connection = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, 
                                                                                         port, 
                                                                                         dbname, 
                                                                                         username, 
                                                                                         pwd))

In [287]:
# read data from database

query_interactions = "select * from user_interaction;"
query_users = "select * from users;"
user_interactions_df = pandas.read_sql_query(query_interactions,connection)
users_dataframe = pandas.read_sql_query(query_users,connection)
# print(user_interactions_dataframe)
# print(users_dataframe)

In [288]:
"""
Task 1 - find events with wrong user_id's
list incorrect events types
"""
# function checks if user_id exist in user list
def check_user_exist(row):
    matching_users = users_dataframe[users_dataframe['user_id']==row.user_id]
    return matching_users.empty
# add column with information which users_id are incorrect
user_interactions_df['incorrect'] = user_interactions_df.apply(check_user_exist, 'columns')
# get dataframe with incorrect events
incorrect_events = user_interactions_df[user_interactions_df['incorrect']]
# get incorrrect events list and print result
print('events on which wrong user id occured {}'.format(incorrect_events['event_type'].drop_duplicates().tolist()))

events on which wrong user id occured ['CLICK', 'VIEW']


In [289]:
"""
Task 2 - find days with high number of duplicated events 
"""
# configuration
# defines what number of duplicated events in one day is listed out in result
min_number_of_duplicates = 1

# add date only column to events list
user_interactions_df['date_only'] = user_interactions_df['event_time'].dt.date
# get all days dataframe
all_days = user_interactions_df['date_only'].drop_duplicates()
all_days = pandas.DataFrame(all_days, columns=['date_only'])
# function which find duplicates
def find_duplicates(row):
    one_day_events = user_interactions_df[user_interactions_df['date_only'] == row.date_only]
    duplicates = one_day_events[one_day_events.duplicated()]
    return(len(duplicates))
# add number of duplicated events to all days dataframe
number_of_duplicates = all_days.apply(find_duplicates, 'columns')
all_days['duplicates'] = number_of_duplicates
# print result
print('days with duplicated events:')
print(all_days[all_days['duplicates']>=min_number_of_duplicates])


days with duplicated events:
    date_only  duplicates
0  2019-05-05           2


In [290]:
"""
Task 3 - find country with greates number of users
Temporary: for tests add more users
"""

extended_users_df = users_dataframe.append(pandas.DataFrame([[101,'18-45', 'PL'], [102,'18-45', 'UK']],
                                        columns = ['user_id', 'age', 'country']))


In [291]:
"""
Task 3 - find countries with greates number of users
Solution
"""
# function that return countires with greates number of users
def max_user_num_from_country(users):
    unique_countries = pandas.DataFrame(users['country'].drop_duplicates(), columns = ['country'])
    unique_countries['num_of_users'] = unique_countries.apply(number_of_users, 'columns', users=users)
    max_number_of_user = unique_countries.max()['num_of_users']
    max_user_num_countries = unique_countries[unique_countries['num_of_users'] == max_number_of_user]
    return max_user_num_countries
# function that return number of usres in given country
def number_of_users(row, users):
    return len(users[users['country'] == row.country])

# print results for users list
print('given list of users:\n{}'.format(users_dataframe))
countries = max_user_num_from_country(users_dataframe)['country'].tolist()
print('\n list of countries with greatest number of users: {}'.format(countries))
# result for data extended for purpose of this exerciese 
print('\n\nextended list of users (created for tests) :\n{}'.format(extended_users_df))
countries = max_user_num_from_country(extended_users_df)['country'].tolist()
print('\n list of countries with greatest number of users: {}'.format(countries))



given list of users:
   user_id    age country
0        3  18-45      DE
1        1  46-80      DE
2       45  46-80      UK
3      100  18-45      UK

 list of countries with greatest number of users: ['DE', 'UK']


extended list of users (created for tests) :
   user_id    age country
0        3  18-45      DE
1        1  46-80      DE
2       45  46-80      UK
3      100  18-45      UK
0      101  18-45      PL
1      102  18-45      UK

 list of countries with greatest number of users: ['UK']


In [292]:
"""
Task 4 
point age groups that performed minimum 5 views and 10 clicks
"""
# configuration
# ditionary  "event_type": needed number of events
events_to_filter = {
    "CLICK": 10,
    "VIEW": 5    
}

# function that return if number of given events for given users is greater or equal than threshold
def calculate_events(row, event_type, threshold):
    user_id = row.user_id
    one_user_interactions = user_interactions_df[user_interactions_df['user_id'] == user_id]
    one_user_interactions_df = pandas.DataFrame(one_user_interactions, columns=['event_type'])
    return len(one_user_interactions_df[one_user_interactions_df['event_type'] == event_type]) >= threshold

# create mask dataframe with filtered informations
users_events_df = pandas.DataFrame([])
for event in events_to_filter:
    users_events_df['{}s'.format(event)] = users_dataframe.apply(calculate_events, 
                                                                 'columns', 
                                                                 event_type=event,
                                                                 threshold = events_to_filter[event]
                                                                )

# function that returns if given row of dataframe exceeds all configured thresholds
def check_all_filters(row):
    return all(row.tolist())
# add mask to users_dataframe and get result
users_dataframe['events_num_mask'] = users_events_df.apply(check_all_filters, 'columns')
result = users_dataframe[users_dataframe['events_num_mask']]['age']

# print result
config_data = ['\n - {}s: {}'.format(event ,events_to_filter[event]) for event in events_to_filter]
config_string = ''.join(config_data)
result_list = result.tolist()
print('Age groups that performed required: {} \n are: {}'.format(config_string, result_list))



Age groups that performed required: 
 - CLICKs: 10
 - VIEWs: 5 
 are: ['18-45', '46-80']
